In [ ]:
# Install dependencies and mount drive
# ----------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import libraries
# ----------------------------------------------------------------------------
import zipfile
import re
import os
import pandas as pd
import numpy as np
from io import BytesIO

print(" Libraries imported successfully")

 Libraries imported successfully


In [ ]:
#Configuration
# ----------------------------------------------------------------------------
# MODIFY THIS PATH to match your Drive location
ZIP_PATH = "/content/drive/MyDrive/CMPE255-NIDSPROJECT/GeneratedLabelledFlows.zip"
OUTPUT_DIR = "/content/drive/MyDrive/CMPE255-NIDSPROJECT/data"

TARGET_FILES = [
    "TrafficLabelling/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",      # DDoS attacks
    "TrafficLabelling/Wednesday-workingHours.pcap_ISCX.csv",                  # Multiple DoS variants
    "TrafficLabelling/Tuesday-WorkingHours.pcap_ISCX.csv"                     # Mixed traffic
]

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f" Output directory: {OUTPUT_DIR}")
print(f" Will process {len(TARGET_FILES)} files")

 Output directory: /content/drive/MyDrive/CMPE255-NIDSPROJECT/data
 Will process 3 files


In [ ]:
# Cell 4: Helper functions
# ----------------------------------------------------------------------------
def normalize_col(c: str) -> str:
    """Trim spaces, remove BOM/nbsp, collapse whitespace."""
    c = (c or "")
    c = c.replace("\ufeff", "").replace("\xa0", " ")
    c = re.sub(r"\s+", " ", c).strip()
    return c

def find_label_col(cols, name_target="label"):
    """Find Label column even if spacing/case differs."""
    for c in cols:
        if normalize_col(c).lower() == name_target.lower():
            return c
    for c in cols:
        if name_target.lower() in normalize_col(c).lower():
            return c
    return None

def is_dos_ddos(label: str) -> bool:
    """Check if label is DoS/DDoS related."""
    if pd.isna(label):
        return False
    label_lower = str(label).lower()
    dos_keywords = ['dos', 'ddos', 'hulk', 'goldeneye', 'slowloris', 'slowhttptest']
    return any(keyword in label_lower for keyword in dos_keywords)

print(" Helper functions defined")

 Helper functions defined


In [ ]:
# Cell 5: Inspect ZIP contents
# ----------------------------------------------------------------------------
with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
    all_csv_files = [f for f in zf.namelist() if f.lower().endswith('.csv')]

print(f" Found {len(all_csv_files)} CSV files in ZIP\n")
print("Files we'll process:")
for i, f in enumerate(TARGET_FILES, 1):
    print(f"  {i}. {f.split('/')[-1]}")


 Found 8 CSV files in ZIP

Files we'll process:
  1. Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
  2. Wednesday-workingHours.pcap_ISCX.csv
  3. Tuesday-WorkingHours.pcap_ISCX.csv


In [ ]:
# Cell 6: Select target CSV file
# ----------------------------------------------------------------------------
# Prioritize files with DoS/DDoS attacks (Wednesday/Friday typically have more attacks)
print("\n" + "="*80)
print("AUTO-DETECTING FILES IN ZIP")
print("="*80)

# First, list ALL files in the ZIP to see the actual structure
with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
    all_files_in_zip = zf.namelist()
    csv_files_in_zip = [f for f in all_files_in_zip if f.lower().endswith('.csv')]

    print(f"✓ Found {len(csv_files_in_zip)} CSV files in ZIP:\n")
    for i, f in enumerate(csv_files_in_zip, 1):
        print(f"  {i}. {f}")

# Now match our target files with actual paths in ZIP
print("\n" + "="*80)
print("MATCHING TARGET FILES")
print("="*80)

target_keywords = [
    "Friday-WorkingHours-Afternoon-DDos",
    "Wednesday-workingHours",
    "Tuesday-WorkingHours"
]

matched_files = []
for keyword in target_keywords:
    for csv_file in csv_files_in_zip:
        if keyword.lower() in csv_file.lower():
            matched_files.append(csv_file)
            print(f"✓ Matched '{keyword}' → {csv_file}")
            break
    else:
        print(f"  Could not find file matching '{keyword}'")

print(f"\n✓ Will process {len(matched_files)} files")

# Now process the matched files
print("\n" + "="*80)
print("PROCESSING MULTIPLE CSV FILES")
print("="*80)

all_dataframes = []

with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
    for idx, csv_file in enumerate(matched_files, 1):
        print(f"\n{'='*80}")
        print(f" Processing File {idx}/{len(matched_files)}: {csv_file.split('/')[-1]}")
        print(f"{'='*80}")

        # Load CSV
        try:
            with zf.open(csv_file) as f:
                df_temp = pd.read_csv(f, low_memory=False)

            print(f" Loaded: {df_temp.shape[0]:,} rows × {df_temp.shape[1]} columns")

            # Normalize column names
            orig_cols = df_temp.columns.tolist()
            norm_cols = [normalize_col(c) for c in orig_cols]
            rename_map = dict(zip(orig_cols, norm_cols))
            df_temp.rename(columns=rename_map, inplace=True)

            # Add source file column for tracking
            file_name = csv_file.split('/')[-1].replace('.pcap_ISCX.csv', '')
            df_temp['source_file'] = file_name

            all_dataframes.append(df_temp)
            print(f"✓ Normalized columns and added source tracking")

        except Exception as e:
            print(f" Error loading {csv_file}: {e}")
            continue

if len(all_dataframes) == 0:
    print("\n ERROR: No files were successfully loaded!")
    print("Please check the ZIP file structure.")
else:
    print(f"\n Successfully loaded {len(all_dataframes)} files")


AUTO-DETECTING FILES IN ZIP
✓ Found 8 CSV files in ZIP:

  1. TrafficLabelling /Wednesday-workingHours.pcap_ISCX.csv
  2. TrafficLabelling /Tuesday-WorkingHours.pcap_ISCX.csv
  3. TrafficLabelling /Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
  4. TrafficLabelling /Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
  5. TrafficLabelling /Monday-WorkingHours.pcap_ISCX.csv
  6. TrafficLabelling /Friday-WorkingHours-Morning.pcap_ISCX.csv
  7. TrafficLabelling /Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
  8. TrafficLabelling /Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv

MATCHING TARGET FILES
✓ Matched 'Friday-WorkingHours-Afternoon-DDos' → TrafficLabelling /Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
✓ Matched 'Wednesday-workingHours' → TrafficLabelling /Wednesday-workingHours.pcap_ISCX.csv
✓ Matched 'Tuesday-WorkingHours' → TrafficLabelling /Tuesday-WorkingHours.pcap_ISCX.csv

✓ Will process 3 files

PROCESSING MULTIPLE CSV FILES

 Processing File 1

In [ ]:
# Cell 7: Load CSV from ZIP
# ----------------------------------------------------------------------------
print(f"\n{'='*80}")
print("COMBINING ALL DATAFRAMES")
print(f"{'='*80}")

# Safety check: ensure we have dataframes to combine
if len(all_dataframes) == 0:
    raise ValueError(" ERROR: No files were successfully loaded! Cannot proceed.")

df = pd.concat(all_dataframes, ignore_index=True)

print(f" Combined {len(all_dataframes)} files")
print(f"  Total shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Show distribution by source file
print("\n Rows per source file:")
print(df['source_file'].value_counts())


COMBINING ALL DATAFRAMES
 Combined 3 files
  Total shape: 1,364,357 rows × 86 columns
  Memory usage: 1351.07 MB

 Rows per source file:
source_file
Wednesday-workingHours                692703
Tuesday-WorkingHours                  445909
Friday-WorkingHours-Afternoon-DDos    225745
Name: count, dtype: int64


CSV files, especially from different sources, often have messy column names with
extra spaces: " Label " or "Flow  Duration" , Special invisible characters: BOM (Byte Order Mark) \ufeff, non-breaking spaces \xa0 , Inconsistent spacing: "Total Fwd Packets" vs "Total  Fwd  Packets"
These issues cause problems when you try to access columns later.
1. Get original column names - This creates a list of all column names as they currently exist in the DataFrame.
2. Normalize Each Column Name - For example , ' Source IP'→'Source IP' , 'Destination Port '→ 'Destination Port'
3. Create a Mapping Dictionary - This tells pandas: Replace old name (key) with new name (value).
4. Rename the DataFrame Columns

In [ ]:
# Cell 8: Normalize column names
# ----------------------------------------------------------------------------
print("\n Normalizing column names...")

orig_cols = df.columns.tolist()
norm_cols = [normalize_col(c) for c in orig_cols]
rename_map = dict(zip(orig_cols, norm_cols))
df.rename(columns=rename_map, inplace=True)

print(f" Column names normalized")

print("\n" + "="*80)
print("ALL COLUMN NAMES")
print("="*80)

print(f"Total columns: {len(df.columns)}\n")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")


 Normalizing column names...
 Column names normalized

ALL COLUMN NAMES
Total columns: 86

   1. Flow ID
   2. Source IP
   3. Source Port
   4. Destination IP
   5. Destination Port
   6. Protocol
   7. Timestamp
   8. Flow Duration
   9. Total Fwd Packets
  10. Total Backward Packets
  11. Total Length of Fwd Packets
  12. Total Length of Bwd Packets
  13. Fwd Packet Length Max
  14. Fwd Packet Length Min
  15. Fwd Packet Length Mean
  16. Fwd Packet Length Std
  17. Bwd Packet Length Max
  18. Bwd Packet Length Min
  19. Bwd Packet Length Mean
  20. Bwd Packet Length Std
  21. Flow Bytes/s
  22. Flow Packets/s
  23. Flow IAT Mean
  24. Flow IAT Std
  25. Flow IAT Max
  26. Flow IAT Min
  27. Fwd IAT Total
  28. Fwd IAT Mean
  29. Fwd IAT Std
  30. Fwd IAT Max
  31. Fwd IAT Min
  32. Bwd IAT Total
  33. Bwd IAT Mean
  34. Bwd IAT Std
  35. Bwd IAT Max
  36. Bwd IAT Min
  37. Fwd PSH Flags
  38. Bwd PSH Flags
  39. Fwd URG Flags
  40. Bwd URG Flags
  41. Fwd Header Length
  42. Bwd H

In [ ]:
# Quick verification cell (optional - run between Cell 8 and 9)
print(f"Total columns after normalization: {len(df.columns)}")
print(f"\n Sample of normalized columns:")
for col in df.columns[:5]:
    print(f"  '{col}' - Length: {len(col)}, Has spaces at ends: {col != col.strip()}")

print(f"\n Label column check:")
label_col = find_label_col(df.columns, "label")
if label_col:
    print(f"  Found: '{label_col}'")
else:
    print("   Not found - showing columns with 'label' in name:")
    for col in df.columns:
        if 'label' in col.lower():
            print(f"    - '{col}'")

Total columns after normalization: 86

 Sample of normalized columns:
  'Flow ID' - Length: 7, Has spaces at ends: False
  'Source IP' - Length: 9, Has spaces at ends: False
  'Source Port' - Length: 11, Has spaces at ends: False
  'Destination IP' - Length: 14, Has spaces at ends: False
  'Destination Port' - Length: 16, Has spaces at ends: False

 Label column check:
  Found: 'Label'


In [ ]:
# Cell 9: Initial data inspection
# ----------------------------------------------------------------------------
print("\n Initial Data Inspection")
print("="*80)
print(f"Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nData types distribution:")
print(df.dtypes.value_counts())


 Initial Data Inspection
Dataset shape: 1,364,357 rows × 86 columns
Memory usage: 1351.07 MB

Data types distribution:
int64      43
float64    37
object      6
Name: count, dtype: int64


In [ ]:
# Cell 10: Check for missing values
# ----------------------------------------------------------------------------
print("\n🔍 Checking Missing Values Across ALL Columns...")
print("="*80)

missing_counts = df.isnull().sum()
missing_percent = (missing_counts / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_counts.index,
    'Missing_Count': missing_counts.values,
    'Missing_Percent': missing_percent.values
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percent', ascending=False)

if len(missing_df) > 0:
    print(f"  Found {len(missing_df)} columns with missing values:\n")
    print(missing_df.to_string(index=False))

    # Visualize missing data severity
    print("\n Missing Data Severity:")
    severe = missing_df[missing_df['Missing_Percent'] > 50]
    moderate = missing_df[(missing_df['Missing_Percent'] > 10) & (missing_df['Missing_Percent'] <= 50)]
    minor = missing_df[missing_df['Missing_Percent'] <= 10]

    print(f"   Severe (>50%): {len(severe)} columns")
    print(f"   Moderate (10-50%): {len(moderate)} columns")
    print(f"   Minor (<10%): {len(minor)} columns")
else:
    print(" No missing values found!")


🔍 Checking Missing Values Across ALL Columns...
  Found 1 columns with missing values:

      Column  Missing_Count  Missing_Percent
Flow Bytes/s           1213         0.088906

 Missing Data Severity:
   Severe (>50%): 0 columns
   Moderate (10-50%): 0 columns
   Minor (<10%): 1 columns


In [ ]:
# Cell 11: Handle missing values
# ----------------------------------------------------------------------------
print("\n  Handling Missing Values...")
print("="*80)

initial_rows = len(df)
initial_cols = len(df.columns)

# Strategy 1: Drop columns with >50% missing values
high_missing_cols = missing_df[missing_df['Missing_Percent'] > 50]['Column'].tolist()
if high_missing_cols:
    print(f"\n Dropping {len(high_missing_cols)} columns with >50% missing:")
    for col in high_missing_cols:
        pct = missing_df[missing_df['Column'] == col]['Missing_Percent'].values[0]
        print(f"  - {col} ({pct:.1f}% missing)")
    df.drop(columns=high_missing_cols, inplace=True)

# Strategy 2: For remaining columns with missing values
remaining_missing = df.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]

if len(remaining_missing) > 0:
    print(f"\n Handling {len(remaining_missing)} columns with moderate/minor missing values:")

    for col in remaining_missing.index:
        missing_count = remaining_missing[col]
        missing_pct = (missing_count / len(df)) * 100

        if df[col].dtype in ['float64', 'int64']:
            # Fill numeric columns with median (using proper assignment to avoid FutureWarning)
            median_val = df[col].median()
            df[col] = df[col].fillna(median_val)
            print(f"  ✓ {col}: Filled with median {median_val:.2f} ({missing_pct:.2f}% missing)")
        else:
            # For non-numeric, drop rows if <1%, else fill with 'Unknown'
            if missing_pct < 1.0:
                df.dropna(subset=[col], inplace=True)
                print(f"   {col}: Dropped {missing_count} rows ({missing_pct:.3f}% missing)")
            else:
                df[col].fillna('Unknown', inplace=True)
                print(f"   {col}: Filled with 'Unknown' ({missing_pct:.2f}% missing)")

print(f"\n Missing value handling complete")
print(f"  Rows: {initial_rows:,} → {len(df):,} ({initial_rows - len(df):,} removed)")
print(f"  Columns: {initial_cols} → {len(df.columns)} ({initial_cols - len(df.columns)} removed)")



  Handling Missing Values...

 Handling 1 columns with moderate/minor missing values:
  ✓ Flow Bytes/s: Filled with median 1039.09 (0.09% missing)

 Missing value handling complete
  Rows: 1,364,357 → 1,364,357 (0 removed)
  Columns: 86 → 86 (0 removed)


In [ ]:
## Cell 12: Check for duplicate rows with diagnostics
# ----------------------------------------------------------------------------
print("\n🔍 Checking for Duplicate Rows...")
print("="*80)

initial_rows = len(df)

# Exclude identifier columns that should be unique
exclude_cols = ['Flow ID', 'Timestamp', 'source_file']
cols_to_check = [c for c in df.columns if c not in exclude_cols]

print(f"Total columns in dataset: {len(df.columns)}")
print(f"Checking duplicates on: {len(cols_to_check)} feature columns")
print(f"Excluded from check: {', '.join(exclude_cols)}")

# === DIAGNOSTIC: Check column types ===
print("\n Data Type Distribution in Columns Being Checked:")
dtype_counts = df[cols_to_check].dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"  • {dtype}: {count} columns")

# === Try 1: Check with exact matching (including float precision) ===
print("\n Attempt 1: Exact matching (including float precision)...")
duplicate_count_exact = df.duplicated(subset=cols_to_check).sum()
print(f"  Duplicates found: {duplicate_count_exact:,} ({duplicate_count_exact/initial_rows*100:.2f}%)")

# === Try 2: Round float columns to reduce precision sensitivity ===
print("\n Attempt 2: Rounding floats to 6 decimals (ML-friendly precision)...")
df_temp = df.copy()
float_cols = df_temp[cols_to_check].select_dtypes(include=['float64']).columns.tolist()
print(f"  Rounding {len(float_cols)} float columns...")

for col in float_cols:
    df_temp[col] = df_temp[col].round(6)

duplicate_count_rounded = df_temp.duplicated(subset=cols_to_check).sum()
print(f"  Duplicates found after rounding: {duplicate_count_rounded:,} ({duplicate_count_rounded/initial_rows*100:.2f}%)")

# === Decision: Use the rounded approach if it finds more duplicates ===
if duplicate_count_rounded > duplicate_count_exact:
    print("\n Using rounded approach (more appropriate for ML)")
    duplicate_count = duplicate_count_rounded
    # Apply rounding to original dataframe
    for col in float_cols:
        df[col] = df[col].round(6)
    print(f"  Applied rounding to {len(float_cols)} float columns in main dataframe")
else:
    print("\n✓ Using exact matching (rounding didn't find additional duplicates)")
    duplicate_count = duplicate_count_exact

# === Process duplicates ===
print(f"\n{'='*80}")
print(f"FINAL RESULT: {duplicate_count:,} duplicates ({duplicate_count/initial_rows*100:.2f}%)")
print(f"{'='*80}")

if duplicate_count > 0:
    print("\n Sample duplicate rows (showing key features + timestamps):")
    duplicates_mask = df.duplicated(subset=cols_to_check, keep=False)
    duplicates = df[duplicates_mask]

    # Show key columns
    display_cols = [c for c in ['Timestamp', 'Flow ID', 'Source IP', 'Destination IP',
                                  'Source Port', 'Destination Port', 'Protocol',
                                  'Flow Duration', 'Total Fwd Packets',
                                  'Total Backward Packets', 'Label', 'source_file']
                    if c in df.columns]
    print(duplicates[display_cols].head(10).to_string())

    # Verify: Check if duplicates have different timestamps
    if 'Timestamp' in df.columns:
        dup_groups = duplicates.groupby(cols_to_check)['Timestamp'].nunique()
        same_timestamp_groups = (dup_groups == 1).sum()
        diff_timestamp_groups = (dup_groups > 1).sum()

        print(f"\n Duplicate Analysis:")
        print(f"   Groups with SAME timestamp: {same_timestamp_groups} (likely data errors)")
        print(f"   Groups with DIFFERENT timestamps: {diff_timestamp_groups} (repeated flow patterns)")

    # Remove duplicates
    print(f"\n  Removing duplicates (keeping first occurrence)...")
    df.drop_duplicates(subset=cols_to_check, inplace=True, keep='first')
    print(f" Removed {duplicate_count:,} duplicate rows")
    print(f"  Rationale: Feature-identical flows are redundant for ML training")
    print(f"  Final shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
else:
    print("\n Analysis: Zero duplicates could mean:")
    print("  1. CIC IDS 2017 has genuinely unique flows (high-quality dataset)")
    print("  2. Timestamp/Flow ID variations make each row unique")
    print("  3. High float precision creates artificial uniqueness")
    print("\n Proceeding with full dataset - no duplicates to remove")
    print(f"  Final shape: {df.shape[0]:,} rows × {df.shape[1]} columns")


🔍 Checking for Duplicate Rows...
Total columns in dataset: 86
Checking duplicates on: 83 feature columns
Excluded from check: Flow ID, Timestamp, source_file

 Data Type Distribution in Columns Being Checked:
  • int64: 43 columns
  • float64: 37 columns
  • object: 3 columns

 Attempt 1: Exact matching (including float precision)...
  Duplicates found: 9,880 (0.72%)

 Attempt 2: Rounding floats to 6 decimals (ML-friendly precision)...
  Rounding 37 float columns...
  Duplicates found after rounding: 9,880 (0.72%)

✓ Using exact matching (rounding didn't find additional duplicates)

FINAL RESULT: 9,880 duplicates (0.72%)

 Sample duplicate rows (showing key features + timestamps):
          Timestamp                                  Flow ID      Source IP  Destination IP  Source Port  Destination Port  Protocol  Flow Duration  Total Fwd Packets  Total Backward Packets   Label                         source_file
405   7/7/2017 3:30  192.168.10.255-192.168.10.17-137-137-17  192.168.10.1

In [ ]:
# Cell 13: Verify label column and display distribution
# ----------------------------------------------------------------------------
print("\n  Label Distribution Analysis")
print("="*80)

label_col = find_label_col(df.columns, "label")

if label_col:
    print(f" Found label column: '{label_col}'\n")

    # Overall label distribution
    label_counts = df[label_col].value_counts()
    print("All labels in dataset:")
    for label, count in label_counts.items():
        pct = count / len(df) * 100
        is_dos = " DoS/DDoS" if is_dos_ddos(label) else ""
        print(f"  {label:40s}: {count:10,} ({pct:6.2f}%) {is_dos}")

    # DoS/DDoS specific count
    dos_count = df[label_col].apply(is_dos_ddos).sum()
    benign_count = df[label_col].str.strip().str.lower().eq('benign').sum()

    print(f"\n Attack Type Summary:")
    print(f"   DoS/DDoS: {dos_count:,} ({dos_count/len(df)*100:.2f}%)")
    print(f"   BENIGN: {benign_count:,} ({benign_count/len(df)*100:.2f}%)")
    print(f"   Other attacks: {len(df) - dos_count - benign_count:,}")

    # Distribution by source file
    print(f"\n Label distribution by source file:")
    file_label_dist = pd.crosstab(df['source_file'], df[label_col])
    print(file_label_dist)

else:
    print("  No label column found!")
    print("Available columns:", df.columns.tolist())


  Label Distribution Analysis
 Found label column: 'Label'

All labels in dataset:
  BENIGN                                  :    967,555 ( 71.43%) 
  DoS Hulk                                :    223,488 ( 16.50%)  DoS/DDoS
  DDoS                                    :    128,027 (  9.45%)  DoS/DDoS
  DoS GoldenEye                           :     10,293 (  0.76%)  DoS/DDoS
  FTP-Patator                             :      7,938 (  0.59%) 
  SSH-Patator                             :      5,897 (  0.44%) 
  DoS slowloris                           :      5,769 (  0.43%)  DoS/DDoS
  DoS Slowhttptest                        :      5,499 (  0.41%)  DoS/DDoS
  Heartbleed                              :         11 (  0.00%) 

 Attack Type Summary:
   DoS/DDoS: 373,076 (27.54%)
   BENIGN: 967,555 (71.43%)
   Other attacks: 13,846

 Label distribution by source file:
Label                               BENIGN    DDoS  DoS GoldenEye  DoS Hulk  \
source_file                                            

In [ ]:
# Cell 14: Save cleaned data (Step 1 complete)
# ----------------------------------------------------------------------------
print("\n Saving Cleaned Data (Missing & Duplicates Handled)...")
print("="*80)

# Dynamic filename based on number of files processed
num_files = df['source_file'].nunique()
output_file = os.path.join(OUTPUT_DIR, f"step1_cleaned_combined_{num_files}files.parquet")
df.to_parquet(output_file, index=False, engine='pyarrow', compression='snappy')

print(f" Saved to: {output_file}")
print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

file_size_mb = os.path.getsize(output_file) / 1024**2
print(f"  File size: {file_size_mb:.2f} MB")

# Also save a metadata summary
metadata = {
    'files_processed': matched_files,
    'total_rows': len(df),
    'total_columns': len(df.columns),
    'source_distribution': df['source_file'].value_counts().to_dict()
}
metadata_file = os.path.join(OUTPUT_DIR, f"step1_metadata_{num_files}files.txt")
with open(metadata_file, 'w') as f:
    f.write("="*80 + "\n")
    f.write("PREPROCESSING STEP 1 METADATA\n")
    f.write("="*80 + "\n\n")
    f.write(f"Files Processed:\n")
    for i, file in enumerate(metadata['files_processed'], 1):
        f.write(f"  {i}. {file}\n")
    f.write(f"\nTotal Rows: {metadata['total_rows']:,}\n")
    f.write(f"Total Columns: {metadata['total_columns']}\n")
    f.write(f"\nRows per Source File:\n")
    for source, count in metadata['source_distribution'].items():
        f.write(f"  {source}: {count:,}\n")

print(f" Saved metadata to: {metadata_file}")



 Saving Cleaned Data (Missing & Duplicates Handled)...
 Saved to: /content/drive/MyDrive/CMPE255-NIDSPROJECT/data/step1_cleaned_combined_3files.parquet
  Shape: 1,354,477 rows × 86 columns
  File size: 181.27 MB
 Saved metadata to: /content/drive/MyDrive/CMPE255-NIDSPROJECT/data/step1_metadata_3files.txt


 Saved interim output as Parquet for:
1. Fast reload if Colab disconnects (avoid re-running 10+ min cleaning)
2. Parquet = 60% smaller + 10x faster than CSV + preserves data types
3. Reproducibility - versioned checkpoints for each preprocessing step
4. Experimentation - try different approaches on same clean baseline
5. Best practice - industry standard for ML pipelines

In [ ]:
# Cell 15: Summary statistics
# ----------------------------------------------------------------------------
print("\n Final Dataset Summary (Step 1 Complete)")
print("="*80)
print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")
print(f"Files combined: {df['source_file'].nunique()}")
print(f"Files: {', '.join(df['source_file'].unique())}")
print(f"\nNumeric columns summary:")
print(df.describe())

print("\n Step 1 Complete: Missing values and duplicates handled!")
print(f" Combined {len(matched_files)} CSV files focused on DoS/DDoS attacks")
print("Next step: Outlier detection and feature engineering")


 Final Dataset Summary (Step 1 Complete)
Total rows: 1,354,477
Total columns: 86
Files combined: 3
Files: Friday-WorkingHours-Afternoon-DDos, Wednesday-workingHours, Tuesday-WorkingHours

Numeric columns summary:
        Source Port  Destination Port      Protocol  Flow Duration  \
count  1.354477e+06      1.354477e+06  1.354477e+06   1.354477e+06   
mean   4.119804e+04      7.308700e+03  9.517292e+00   2.057618e+07   
std    2.172930e+04      1.786470e+04  5.135087e+00   3.805836e+07   
min    0.000000e+00      0.000000e+00  0.000000e+00  -4.000000e+00   
25%    3.291600e+04      5.300000e+01  6.000000e+00   2.280000e+02   
50%    5.064500e+04      8.000000e+01  6.000000e+00   6.943300e+04   
75%    5.791000e+04      4.430000e+02  1.700000e+01   9.332258e+06   
max    6.553500e+04      6.553200e+04  1.700000e+01   1.200000e+08   

       Total Fwd Packets  Total Backward Packets  Total Length of Fwd Packets  \
count       1.354477e+06            1.354477e+06                 1.354477e

In [ ]:
# Outlier detection and Removal - Import libraries and load Step 1 output
import pandas as pd
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')

data_dir = '/content/drive/MyDrive/CMPE255-NIDSPROJECT/data'
print(f"Checking directory: {data_dir}")


print("\n" + "="*80)
print("STEP 2: OUTLIER DETECTION & REMOVAL")
print("="*80)

print("\n Loading Step 1 cleaned data...")
df = pd.read_parquet('/content/drive/MyDrive/CMPE255-NIDSPROJECT/data/step1_cleaned_combined_3files.parquet')

print(f" Loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

initial_rows = len(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checking directory: /content/drive/MyDrive/CMPE255-NIDSPROJECT/data

STEP 2: OUTLIER DETECTION & REMOVAL

 Loading Step 1 cleaned data...
 Loaded: 1,354,477 rows × 86 columns
Memory usage: 1341.30 MB


In [ ]:
#Identify numeric columns for outlier detection
print("\n" + "="*80)
print("IDENTIFYING NUMERIC COLUMNS")
print("="*80)

# Get numeric columns (exclude ID, timestamp, categorical)
exclude_cols = ['Flow ID', 'Timestamp', 'Source IP', 'Destination IP', 'Label', 'source_file']
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

print(f"Total numeric columns for outlier detection: {len(numeric_cols)}")
print(f"\nFirst 15 numeric columns:")
for i, col in enumerate(numeric_cols[:15], 1):
    print(f"  {i:2d}. {col}")


IDENTIFYING NUMERIC COLUMNS
Total numeric columns for outlier detection: 80

First 15 numeric columns:
   1. Source Port
   2. Destination Port
   3. Protocol
   4. Flow Duration
   5. Total Fwd Packets
   6. Total Backward Packets
   7. Total Length of Fwd Packets
   8. Total Length of Bwd Packets
   9. Fwd Packet Length Max
  10. Fwd Packet Length Min
  11. Fwd Packet Length Mean
  12. Fwd Packet Length Std
  13. Bwd Packet Length Max
  14. Bwd Packet Length Min
  15. Bwd Packet Length Mean


In [ ]:
#Detect and fix negative values <<TBD>>
import pandas as pd
import numpy as np
import os

print("\n" + "="*80)
print("STEP 2: OUTLIER DETECTION & REMOVAL")
print("="*80)

print("\n Loading Step 1 cleaned data...")
df = pd.read_parquet('/content/drive/MyDrive/CMPE255-NIDSPROJECT/data/step1_cleaned_combined_3files.parquet')

print(f" Loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

initial_rows = len(df)

print("\n" + "="*80)
print("STEP 2.1: DETECTING DOMAIN VIOLATIONS (Negative Values)")
print("="*80)

# Features that CANNOT be negative (domain knowledge)
must_be_positive = [
    'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
    'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
    'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
    'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean',
    'Flow Bytes/s', 'Flow Packets/s', 'Fwd Packets/s', 'Bwd Packets/s',
    'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
    'min_seg_size_forward', 'Active Mean', 'Active Max', 'Active Min',
    'Idle Mean', 'Idle Max', 'Idle Min'
]

# Find columns that exist in dataframe
must_be_positive = [col for col in must_be_positive if col in df.columns]

print(f"Checking {len(must_be_positive)} columns that must be non-negative:\n")

negative_issues = {}
for col in must_be_positive:
    negative_count = (df[col] < 0).sum()
    if negative_count > 0:
        negative_issues[col] = negative_count
        negative_pct = (negative_count / len(df)) * 100
        print(f"  {col}: {negative_count:,} negative values ({negative_pct:.3f}%)")

if negative_issues:
    print(f"\n Fixing {len(negative_issues)} columns with negative values...")

    for col in negative_issues.keys():
        # Strategy: Remove rows with negative values (data quality issues)
        rows_before = len(df)
        df = df[df[col] >= 0]
        rows_removed = rows_before - len(df)
        print(f"   {col}: Removed {rows_removed:,} rows")

    print(f"\n Domain violations fixed")
    print(f"  Rows: {initial_rows:,} → {len(df):,} ({initial_rows - len(df):,} removed)")
else:
    print("\n No negative values found in columns that must be positive!")



STEP 2: OUTLIER DETECTION & REMOVAL

 Loading Step 1 cleaned data...
 Loaded: 1,354,477 rows × 86 columns
Memory usage: 1341.30 MB

STEP 2.1: DETECTING DOMAIN VIOLATIONS (Negative Values)
Checking 23 columns that must be non-negative:

  Flow Duration: 40 negative values (0.003%)
  Flow Bytes/s: 18 negative values (0.001%)
  Flow Packets/s: 40 negative values (0.003%)
  min_seg_size_forward: 12 negative values (0.001%)

 Fixing 4 columns with negative values...
   Flow Duration: Removed 40 rows
   Flow Bytes/s: Removed 0 rows
   Flow Packets/s: Removed 0 rows
   min_seg_size_forward: Removed 12 rows

 Domain violations fixed
  Rows: 1,354,477 → 1,354,425 (52 removed)


In [ ]:
#Detect zero-duration flows <<TBD>>
#A zero-duration flow is a network connection
#where the time difference between the first and last packet is 0 microseconds (or effectively 0).
print("\n" + "="*80)
print("STEP 2.2: HANDLING ZERO-DURATION FLOWS")
print("="*80)

if 'Flow Duration' in df.columns:
    zero_duration = (df['Flow Duration'] == 0).sum()
    zero_duration_pct = (zero_duration / len(df)) * 100

    print(f"Zero-duration flows: {zero_duration:,} ({zero_duration_pct:.2f}%)")

    if zero_duration > 0:
        print("\n Analyzing zero-duration flows:")
        zero_flows = df[df['Flow Duration'] == 0]

        # Check if they have packets
        if 'Total Fwd Packets' in df.columns and 'Total Backward Packets' in df.columns:
            zero_with_packets = ((zero_flows['Total Fwd Packets'] > 0) |
                                 (zero_flows['Total Backward Packets'] > 0)).sum()
            print(f"   With packets: {zero_with_packets:,}")
            print(f"   Without packets: {zero_duration - zero_with_packets:,}")

        # Decision: Remove zero-duration flows (problematic for rate calculations)
        print("\n  Removing zero-duration flows (causes division by zero issues)...")
        rows_before = len(df)
        df = df[df['Flow Duration'] > 0]
        rows_removed = rows_before - len(df)
        print(f"  Removed {rows_removed:,} zero-duration flows")
        print(f"  Rows: {rows_before:,} → {len(df):,}")
else:
    print("  Flow Duration column not found")


STEP 2.2: HANDLING ZERO-DURATION FLOWS
Zero-duration flows: 1,570 (0.12%)

 Analyzing zero-duration flows:
   With packets: 1,570
   Without packets: 0

  Removing zero-duration flows (causes division by zero issues)...
  Removed 1,570 zero-duration flows
  Rows: 1,354,425 → 1,352,855


In [ ]:
# outlier detection using IQR method <<TBD>>
print("\n" + "="*80)
print("STEP 2.3: STATISTICAL OUTLIER DETECTION (IQR Method)")
print("="*80)

print("Using IQR (Interquartile Range) method to detect extreme outliers")
print("Outliers defined as: values < Q1 - 3*IQR or values > Q3 + 3*IQR")
print("(Using 3*IQR instead of 1.5*IQR to be less aggressive)\n")

outlier_summary = []

for col in numeric_cols:
    if col not in df.columns:
        continue

    # Calculate IQR
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    # Define outlier bounds (3*IQR for less aggressive removal)
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR

    # Count outliers
    outliers_lower = (df[col] < lower_bound).sum()
    outliers_upper = (df[col] > upper_bound).sum()
    total_outliers = outliers_lower + outliers_upper

    if total_outliers > 0:
        outlier_pct = (total_outliers / len(df)) * 100
        outlier_summary.append({
            'Column': col,
            'Outliers': total_outliers,
            'Percentage': outlier_pct,
            'Lower_Bound': lower_bound,
            'Upper_Bound': upper_bound,
            'Q1': Q1,
            'Q3': Q3
        })

# Convert to DataFrame and sort by percentage
outlier_df = pd.DataFrame(outlier_summary).sort_values('Percentage', ascending=False)

print(f" Columns with outliers (top 20):\n")
if len(outlier_df) > 0:
    display_df = outlier_df.head(20).copy()
    display_df['Percentage'] = display_df['Percentage'].apply(lambda x: f"{x:.2f}%")
    print(display_df[['Column', 'Outliers', 'Percentage']].to_string(index=False))
else:
    print(" No statistical outliers detected!")



STEP 2.3: STATISTICAL OUTLIER DETECTION (IQR Method)
Using IQR (Interquartile Range) method to detect extreme outliers
Outliers defined as: values < Q1 - 3*IQR or values > Q3 + 3*IQR
(Using 3*IQR instead of 1.5*IQR to be less aggressive)

 Columns with outliers (top 20):

                Column  Outliers Percentage
            Active Min    335417     24.79%
            Active Max    332297     24.56%
           Active Mean    332254     24.56%
        PSH Flag Count    321621     23.77%
         Fwd IAT Total    289375     21.39%
          Flow Bytes/s    284870     21.06%
         Flow Duration    283972     20.99%
         Bwd Packets/s    281952     20.84%
           Bwd IAT Max    281456     20.80%
          Bwd IAT Mean    274297     20.28%
         Bwd IAT Total    272398     20.14%
Packet Length Variance    264638     19.56%
           Bwd IAT Std    262271     19.39%
           Fwd IAT Min    245841     18.17%
          Fwd IAT Mean    224686     16.61%
          Flow IAT Min

In [ ]:
# Remove extreme outliers <<TBD>>
print("\n" + "="*80)
print("STEP 2.4: REMOVING EXTREME OUTLIERS")
print("="*80)

rows_before_outlier_removal = len(df)

# Strategy: Mark rows with outliers in critical columns only
# Not all columns - focus on flow characteristics that matter most for DoS/DDoS

critical_cols = [
    'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
    'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
    'Flow Bytes/s', 'Flow Packets/s'
]
critical_cols = [col for col in critical_cols if col in df.columns]

print(f"Removing outliers from {len(critical_cols)} critical columns:")
for col in critical_cols:
    print(f"   {col}")

outlier_mask = pd.Series([False] * len(df), index=df.index)

for col in critical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR

    # Mark rows with outliers
    col_outliers = (df[col] < lower_bound) | (df[col] > upper_bound)
    outlier_mask = outlier_mask | col_outliers

    if col_outliers.sum() > 0:
        print(f"  {col}: {col_outliers.sum():,} outliers")

# Remove rows marked as outliers
df = df[~outlier_mask]
rows_removed = rows_before_outlier_removal - len(df)
removal_pct = (rows_removed / rows_before_outlier_removal) * 100

print(f"\n Outlier removal complete")
print(f"  Removed: {rows_removed:,} rows ({removal_pct:.2f}%)")
print(f"  Remaining: {len(df):,} rows")



STEP 2.4: REMOVING EXTREME OUTLIERS
Removing outliers from 7 critical columns:
   Flow Duration
   Total Fwd Packets
   Total Backward Packets
   Total Length of Fwd Packets
   Total Length of Bwd Packets
   Flow Bytes/s
   Flow Packets/s
  Flow Duration: 283,972 outliers
  Total Fwd Packets: 66,507 outliers
  Total Backward Packets: 47,248 outliers
  Total Length of Fwd Packets: 99,218 outliers
  Total Length of Bwd Packets: 33,437 outliers
  Flow Bytes/s: 284,870 outliers
  Flow Packets/s: 116,355 outliers

 Outlier removal complete
  Removed: 682,156 rows (50.42%)
  Remaining: 670,699 rows


In [ ]:
#Validate data ranges (domain constraints)
print("\n" + "="*80)
print("STEP 2.5: DOMAIN CONSTRAINT VALIDATION")
print("="*80)

print("Applying domain-specific validation rules:\n")

validation_issues = 0

# Rule 1: Ports should be 0-65535
for port_col in ['Source Port', 'Destination Port']:
    if port_col in df.columns:
        invalid_ports = ((df[port_col] < 0) | (df[port_col] > 65535)).sum()
        if invalid_ports > 0:
            print(f"  {port_col}: {invalid_ports:,} invalid values")
            df = df[(df[port_col] >= 0) & (df[port_col] <= 65535)]
            validation_issues += invalid_ports

# Rule 2: Protocol should be valid (0-255 typically)
if 'Protocol' in df.columns:
    invalid_protocol = ((df['Protocol'] < 0) | (df['Protocol'] > 255)).sum()
    if invalid_protocol > 0:
        print(f"  Protocol: {invalid_protocol:,} invalid values")
        df = df[(df['Protocol'] >= 0) & (df['Protocol'] <= 255)]
        validation_issues += invalid_protocol

# Rule 3: Packet counts should make sense
if 'Total Fwd Packets' in df.columns and 'Total Backward Packets' in df.columns:
    both_zero = ((df['Total Fwd Packets'] == 0) & (df['Total Backward Packets'] == 0)).sum()
    if both_zero > 0:
        print(f"  Flows with zero packets (both directions): {both_zero:,}")
        df = df[~((df['Total Fwd Packets'] == 0) & (df['Total Backward Packets'] == 0))]
        validation_issues += both_zero

if validation_issues > 0:
    print(f"\n Validation complete: Removed {validation_issues:,} rows with constraint violations")
else:
    print(" All domain constraints satisfied!")

print(f"  Current shape: {len(df):,} rows × {len(df.columns)} columns")



STEP 2.5: DOMAIN CONSTRAINT VALIDATION
Applying domain-specific validation rules:

 All domain constraints satisfied!
  Current shape: 670,699 rows × 86 columns


In [ ]:
#Summary statistics after outlier removal <<TBD>>
print("\n" + "="*80)
print("STEP 2 STATISTICS: BEFORE vs AFTER")
print("="*80)

print(f"\nRows removed summary:")
print(f"  Initial (Step 1 output):     {initial_rows:,}")
print(f"  After negative value removal: {len(df):,}")
print(f"  Total removed in Step 2:      {initial_rows - len(df):,} ({(initial_rows - len(df))/initial_rows*100:.2f}%)")
print(f"  Final (Step 2 output):        {len(df):,}")

# Show improved statistics for key columns
print(f"\n Key columns - statistics after cleaning:")
key_cols = ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
            'Flow Bytes/s', 'Flow Packets/s']
key_cols = [col for col in key_cols if col in df.columns]

if key_cols:
    print(df[key_cols].describe())


STEP 2 STATISTICS: BEFORE vs AFTER

Rows removed summary:
  Initial (Step 1 output):     1,354,477
  After negative value removal: 670,699
  Total removed in Step 2:      683,778 (50.48%)
  Final (Step 2 output):        670,699

 Key columns - statistics after cleaning:
       Flow Duration  Total Fwd Packets  Total Backward Packets  \
count   6.706990e+05      670699.000000           670699.000000   
mean    1.965673e+06           2.988645                2.277754   
std     4.443902e+06           2.455554                2.527590   
min     3.300000e+01           1.000000                0.000000   
25%     2.404600e+04           1.000000                1.000000   
50%     6.885800e+04           2.000000                1.000000   
75%     1.357050e+06           4.000000                3.000000   
max     3.749164e+07          18.000000               21.000000   

        Flow Bytes/s  Flow Packets/s  
count  670699.000000   670699.000000  
mean     6497.095200     3591.424811  
std    

In [ ]:
#Check label distribution after outlier removal
print("\n" + "="*80)
print("LABEL DISTRIBUTION AFTER OUTLIER REMOVAL")
print("="*80)

label_col = 'Label'
if label_col in df.columns:
    print("\nLabel counts:")
    label_counts = df[label_col].value_counts()

    for label, count in label_counts.items():
        pct = count / len(df) * 100
        print(f"  {label:30s}: {count:10,} ({pct:6.2f}%)")

    # Check if we still have good representation of DoS/DDoS
    dos_keywords = ['dos', 'ddos', 'hulk', 'goldeneye', 'slowloris', 'slowhttptest']
    dos_count = df[label_col].apply(lambda x: any(k in str(x).lower() for k in dos_keywords)).sum()
    benign_count = df[label_col].str.strip().str.lower().eq('benign').sum()

    print(f"\n Attack Type Summary:")
    print(f"   DoS/DDoS: {dos_count:,} ({dos_count/len(df)*100:.2f}%)")
    print(f"   BENIGN:   {benign_count:,} ({benign_count/len(df)*100:.2f}%)")
    print(f"   Other:    {len(df) - dos_count - benign_count:,}")



LABEL DISTRIBUTION AFTER OUTLIER REMOVAL

Label counts:
  BENIGN                        :    522,004 ( 77.83%)
  DDoS                          :     89,988 ( 13.42%)
  DoS Hulk                      :     38,241 (  5.70%)
  DoS GoldenEye                 :      8,196 (  1.22%)
  FTP-Patator                   :      6,434 (  0.96%)
  SSH-Patator                   :      2,876 (  0.43%)
  DoS slowloris                 :      1,981 (  0.30%)
  DoS Slowhttptest              :        979 (  0.15%)

 Attack Type Summary:
   DoS/DDoS: 139,385 (20.78%)
   BENIGN:   522,004 (77.83%)
   Other:    9,310


In [ ]:
# Save Step 2 output
print("\n" + "="*80)
print("SAVING STEP 2 OUTPUT")
print("="*80)

output_file = os.path.join(OUTPUT_DIR, "step2_cleaned_outliers_removed.parquet")
df.to_parquet(output_file, index=False, engine='pyarrow', compression='snappy')

file_size_mb = os.path.getsize(output_file) / 1024**2

print(f" Saved to: {output_file}")
print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"  File size: {file_size_mb:.2f} MB")

# Save metadata
num_files = df['source_file'].nunique()
metadata_file = os.path.join(OUTPUT_DIR, f"step2_metadata_{num_files}files.txt")
with open(metadata_file, 'w') as f:
    f.write("="*80 + "\n")
    f.write("PREPROCESSING STEP 2 METADATA\n")
    f.write("="*80 + "\n\n")
    f.write(f"Input: step1_cleaned_combined_{num_files}files.parquet\n")
    f.write(f"Output: step2_cleaned_outliers_removed.parquet\n\n")
    f.write(f"Initial Rows (Step 1): {initial_rows:,}\n")
    f.write(f"Final Rows (Step 2):   {len(df):,}\n")
    f.write(f"Rows Removed:          {initial_rows - len(df):,} ({(initial_rows - len(df))/initial_rows*100:.2f}%)\n\n")
    f.write("Outlier Removal Summary:\n")
    f.write(f"   Negative value fixes\n")
    f.write(f"   Zero-duration flow removal\n")
    f.write(f"   IQR-based outlier removal (3*IQR)\n")
    f.write(f"   Domain constraint validation\n")

print(f" Saved metadata to: {metadata_file}")

print("\n" + "="*80)
print(" STEP 2 COMPLETE: OUTLIER DETECTION & REMOVAL")
print("="*80)
print(f"Final dataset: {len(df):,} rows × {len(df.columns)} columns")
print("Ready for Step 3: Feature Engineering")


SAVING STEP 2 OUTPUT
 Saved to: /content/drive/MyDrive/CMPE255-NIDSPROJECT/data/step2_cleaned_outliers_removed.parquet
  Shape: 670,699 rows × 86 columns
  File size: 86.38 MB
 Saved metadata to: /content/drive/MyDrive/CMPE255-NIDSPROJECT/data/step2_metadata_3files.txt

 STEP 2 COMPLETE: OUTLIER DETECTION & REMOVAL
Final dataset: 670,699 rows × 86 columns
Ready for Step 3: Feature Engineering
